# 6. HCC1806 Predictions
We will now use the Ensemble method developed in the training notebook in order to predict the environment of the cells in the test dataset.

In [1]:
import pandas as pd
import pickle
import numpy as np

# 1. Importing the files

In [2]:
tumor = 'MCF7'

In [3]:
df_smartseq_pathHFiltNorm = f'../ai_lab/SmartSeq/{tumor}_SmartS_Filtered_Normalised_3000_Data_test_anonim.txt'
df_smartseq_HFiltNorm = pd.read_csv(df_smartseq_pathHFiltNorm, delimiter=' ')

In [4]:
df_dropseq_pathHFiltNorm = f'../ai_lab/DropSeq/{tumor}_Filtered_Normalised_3000_Data_test_anonim.txt'
df_dropseq_HFiltNorm = pd.read_csv(df_dropseq_pathHFiltNorm, delimiter=' ')

We take a look at the structure of this dataset

In [5]:
df_smartseq_HFiltNorm.head()

,1,2,3,4,5,6,7,8,9,10,...,54,55,56,57,58,59,60,61,62,63
CYP1B1,492,7199,12,373,31,245,258,20634,6804,570,...,202,120,340,7919,2015,287,15220,21998,39,195
CYP1B1-AS1,253,3245,11,187,13,109,106,8769,2911,246,...,89,57,151,3515,929,126,6316,8898,17,81
CYP1A1,0,7181,1,0,0,0,0,4813,72,1,...,0,0,0,0,75,666,1991,21329,1,0
NDRG1,1157,1857,5,0,0,10,3,1196,1168,2,...,9,0,0,503,237,3270,750,1498,29,6
DDIT4,6805,20731,147,43,0,25,646,11080,2988,9,...,1,0,4,5323,16733,25776,12176,5144,20,93


In [6]:
df_dropseq_HFiltNorm.head()

,1,2,3,4,5,6,7,8,9,10,...,5397,5398,5399,5400,5401,5402,5403,5404,5405,5406
MALAT1,0,0,3,0,1,0,6,0,1,0,...,2,0,0,11,2,0,0,1,5,1
MT-RNR2,0,2,1,0,0,0,1,0,0,0,...,11,0,0,0,0,0,0,0,0,0
NEAT1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
H1-5,0,0,0,0,0,1,1,0,0,0,...,0,0,0,3,0,0,0,0,0,0
TFF1,3,5,2,3,3,1,2,6,7,0,...,0,4,1,0,0,3,2,0,0,1


We then take the transpose in order to have cells as rows and features as columns and then look at the structure.

In [7]:
df_smartseq_HFiltNormT = df_smartseq_HFiltNorm.T
df_smartseq_HFiltNormT.head()

,CYP1B1,CYP1B1-AS1,CYP1A1,NDRG1,DDIT4,PFKFB3,HK2,AREG,MYBL2,ADM,...,CD27-AS1,DNAI7,MAFG,LZTR1,BCO2,GRIK5,SLC25A27,DENND5A,CDK5R1,FAM13A-AS1
1,492,253,0,1157,6805,5447,2662,2448,0,58,...,0,0,193,0,0,0,0,285,0,1
2,7199,3245,7181,1857,20731,4374,5490,0,0,2150,...,0,0,187,0,0,0,0,55,0,0
3,12,11,1,5,147,301,9,647,1890,1,...,0,0,28,0,0,0,0,1,0,0
4,373,187,0,0,43,37,0,0,580,0,...,21,0,131,0,0,0,0,3,0,0
5,31,13,0,0,0,44,8,0,554,0,...,43,0,30,0,0,0,0,1,0,0


In [8]:
# we notice that the genes are in a different order than the training dataset, we reorder them based on the training dataset
columns_smartseq_file = f'../ai_lab/SmartSeq/{tumor}_SmartS_Filtered_Normalised_3000_Data_train.txt'
columns_smartseq_df = pd.read_csv(columns_smartseq_file, delimiter=' ')
columns_smartseq = columns_smartseq_df.T.columns
df_smartseq_HFiltNormT = df_smartseq_HFiltNormT[columns_smartseq]

In [9]:
df_smartseq_HFiltNormT.head()

,CYP1B1,CYP1B1-AS1,CYP1A1,NDRG1,DDIT4,PFKFB3,HK2,AREG,MYBL2,ADM,...,CD27-AS1,DNAI7,MAFG,LZTR1,BCO2,GRIK5,SLC25A27,DENND5A,CDK5R1,FAM13A-AS1
1,492,253,0,1157,6805,5447,2662,2448,0,58,...,0,0,193,0,0,0,0,285,0,1
2,7199,3245,7181,1857,20731,4374,5490,0,0,2150,...,0,0,187,0,0,0,0,55,0,0
3,12,11,1,5,147,301,9,647,1890,1,...,0,0,28,0,0,0,0,1,0,0
4,373,187,0,0,43,37,0,0,580,0,...,21,0,131,0,0,0,0,3,0,0
5,31,13,0,0,0,44,8,0,554,0,...,43,0,30,0,0,0,0,1,0,0


In [10]:
df_dropseq_HFiltNormT = df_dropseq_HFiltNorm.T
df_dropseq_HFiltNormT.head()

,MALAT1,MT-RNR2,NEAT1,H1-5,TFF1,MT-RNR1,H4C3,GDF15,KRT81,MT-CO3,...,MROH1,SKIDA1,MICALL1,RARG,MYO1F,BRWD1-AS2,RPS19BP1,AUNIP,TNK2,SUDS3
1,0,0,0,0,3,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,2,0,0,5,0,3,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,3,1,0,0,2,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,3,0,2,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5,1,0,0,0,3,0,8,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [13]:
# we notice that the genes are in a different order than the training dataset, we reorder them based on the training dataset
columns_dropseq_file =  f'../ai_lab/DropSeq/{tumor}_Filtered_Normalised_3000_Data_train.txt'
columns_dropseq_df = pd.read_csv(columns_dropseq_file, delimiter=' ')
columns_dropseq = columns_dropseq_df.T.columns
df_dropseq_HFiltNormT = df_dropseq_HFiltNormT[columns_dropseq]

In [14]:
df_dropseq_HFiltNormT.head()

,MALAT1,MT-RNR2,NEAT1,H1-5,TFF1,MT-RNR1,H4C3,GDF15,KRT81,MT-CO3,...,MROH1,SKIDA1,MICALL1,RARG,MYO1F,BRWD1-AS2,RPS19BP1,AUNIP,TNK2,SUDS3
1,0,0,0,0,3,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,2,0,0,5,0,3,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,3,1,0,0,2,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,3,0,2,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5,1,0,0,0,3,0,8,0,0,0,...,0,0,0,0,0,0,0,0,0,0


# Ensemble method

In [15]:
names = ['logreg', 'nn', 'xgboost']

In [16]:
class Ensemble():
    def __init__(self, models):
        self.models = models

    def predict(self, X):
        predictions = np.column_stack([model.predict(X) for model in self.models.values()])
        return np.mean(predictions, axis=1).astype(int)

    def score(self, X, y):
        return accuracy_score(y, self.predict(X))

In [17]:
models = {}
X = df_dropseq_HFiltNormT

for name in names:
    print(name)
    filename = f"pickle/{tumor}/{name}/dropseq_merged dataset"
    with open(filename, 'rb') as f:
        result = pickle.load(f)
        models[name] = result['model']


ensemble_model = Ensemble(models)

# Make predictions
ensemble_predictions = ensemble_model.predict(X)
prediction_dropseq_df = df_dropseq_HFiltNormT.copy()
prediction_dropseq_df['Prediction'] = ensemble_predictions
prediction_dropseq_df['Prediction'] = prediction_dropseq_df['Prediction'].apply(lambda x: 'Normoxia' if x == 1 else 'Hypoxia')
prediction_dropseq_df.head()

logreg
nn
xgboost


,MALAT1,MT-RNR2,NEAT1,H1-5,TFF1,MT-RNR1,H4C3,GDF15,KRT81,MT-CO3,...,SKIDA1,MICALL1,RARG,MYO1F,BRWD1-AS2,RPS19BP1,AUNIP,TNK2,SUDS3,Prediction
1,0,0,0,0,3,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,Hypoxia
2,0,2,0,0,5,0,3,0,0,0,...,0,0,0,0,0,0,0,0,0,Normoxia
3,3,1,0,0,2,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,Normoxia
4,0,0,0,0,3,0,2,0,0,0,...,0,0,0,0,0,0,0,0,0,Hypoxia
5,1,0,0,0,3,0,8,0,0,0,...,0,0,0,0,0,0,0,0,0,Hypoxia


In [18]:
models = {}
X = df_smartseq_HFiltNormT

for name in names:
    print(name)
    filename = f"pickle/{tumor}/{name}/smartseq_merged dataset"
    with open(filename, 'rb') as f:
        result = pickle.load(f)
        models[name] = result['model']


ensemble_model = Ensemble(models)

# Make predictions
ensemble_predictions = ensemble_model.predict(X)
prediction_smartseq_df = df_smartseq_HFiltNormT.copy()
prediction_smartseq_df['Prediction'] = ensemble_predictions
prediction_smartseq_df['Prediction'] = prediction_smartseq_df['Prediction'].apply(lambda x: 'Normoxia' if x == 1 else 'Hypoxia')
prediction_smartseq_df.head()

logreg
nn
xgboost


,CYP1B1,CYP1B1-AS1,CYP1A1,NDRG1,DDIT4,PFKFB3,HK2,AREG,MYBL2,ADM,...,DNAI7,MAFG,LZTR1,BCO2,GRIK5,SLC25A27,DENND5A,CDK5R1,FAM13A-AS1,Prediction
1,492,253,0,1157,6805,5447,2662,2448,0,58,...,0,193,0,0,0,0,285,0,1,Hypoxia
2,7199,3245,7181,1857,20731,4374,5490,0,0,2150,...,0,187,0,0,0,0,55,0,0,Hypoxia
3,12,11,1,5,147,301,9,647,1890,1,...,0,28,0,0,0,0,1,0,0,Normoxia
4,373,187,0,0,43,37,0,0,580,0,...,0,131,0,0,0,0,3,0,0,Normoxia
5,31,13,0,0,0,44,8,0,554,0,...,0,30,0,0,0,0,1,0,0,Normoxia


In [19]:
# export the prediction dataframes to csv
prediction_dropseq_df.to_csv(f'../ai_lab/predictions/DropSeq{tumor}.csv')
prediction_smartseq_df.to_csv(f'../ai_lab/predictions/SmartSeq{tumor}.csv')